In [1]:
import os
import librosa
import numpy as np
import pandas as pd
import faiss



Traitement des audio et extraction des caractéristiques audios pertinentes (mfcc)

In [2]:

def feature_extraction(file_path):
    try:
        # load the audio file
        x, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        # extract features from the audio
        mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=50).T, axis=0)
        return mfcc
    except ValueError as e:
        # Gérer l'erreur ici (par exemple, imprimer un message d'erreur ou ignorer le fichier)
        print(f"Erreur lors du chargement de {file_path}: {e}")
        return None

directory = 'C:/Users/femia/Desktop/dogs/'
features = []

for audio in os.listdir(directory):
    audio_path = os.path.join(directory, audio)
    if os.path.getsize(audio_path) > 0:  # Vérifiez si le fichier n'est pas vide
        mfccs = feature_extraction(audio_path)
        if mfccs is not None:
            features.append(mfccs)


Stockage des noms d'animaux

In [3]:
# Liste pour stocker les noms des animaux
names = []

# Parcourir le répertoire et ajouter les noms des fichiers à la liste
for filename in os.listdir(directory):
    if os.path.isfile(os.path.join(directory, filename)):
        names.append(filename)




Fonction de création des index, de recherche à l'intérieur des index

In [4]:
def creatingIndexes(ensemble):
    d = ensemble.shape[1]
    index = faiss.IndexFlatL2(d)  # Utiliser l'indexation L2
    index.add(ensemble)
    return index


In [5]:
def searchinIndex(index, normal_embedding):
    D,I= index.search(normal_embedding, 10)
    r=pd.DataFrame({'distance':D[0],'index':I[0]})
    return r

In [6]:
embedding=np.array(features,dtype=np.float32)

In [7]:
index=creatingIndexes(embedding)

In [8]:
query_embedding=feature_extraction("C:/Users/femia/Downloads/output_12_15_59.wav")
# Assurez-vous que `query_embedding` est un tableau 2D avec une seule requête
query_embedding = query_embedding.reshape(1, -1)
query_embedding=np.array(query_embedding,dtype=np.float32)

In [9]:
i=searchinIndex(index,query_embedding)

In [10]:
print(i)

      distance  index
0  2520.647217      9
1  2528.609863    594
2  2683.003174    545
3  2723.091064    574
4  2726.721680    325
5  2760.156982      0
6  2776.731689    563
7  2778.452148    606
8  2786.254639    139
9  2802.515137     52


Nom de l'espèce 

In [11]:
print("L'animal est un " + names[i['index'][0]].replace('.aif', ''))


L'animal est un Farley-1-P-1b
